In [2]:
import pandas as pd
import json

from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import Qdrant

import qdrant_client
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

# Load API key.
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
with open('salesItem.csv') as f:
    for _ in range(5):
        print(f.readline())


"SalesItemId";"SalesItemCode";"SalesItemName";"ActiveIngredientsName";"Formularium";"OrganizationId"

"19000000062";"MCD00062";"2 MTM BODY LOTION (DR MUTTY)";" KOSMETIK";"NORMAL";"19"

"100000000146092";"KJD00002";"2-4 (ZWAVEL ZALF) 30GRAM - N/A";" SULFUR, SALICYLIC ACID";"NORMAL";"3,31"

"100000000146091";"KJD00001";"2-4 ZALF /GRAM - N/A";" SULFUR, SALICYLIC ACID";"NORMAL";"3,5,31"

"100000000142499";"DT00146R";"3TC 150MG TAB";" LAMIVUDINE";"NORMAL";"3,17,27,32,43"



In [4]:
PATH = 'salesItem.csv'
df = pd.read_csv(PATH, sep=';')
df.head()


,SalesItemId,SalesItemCode,SalesItemName,ActiveIngredientsName,Formularium,OrganizationId
0,19000000062,MCD00062,2 MTM BODY LOTION (DR MUTTY),KOSMETIK,NORMAL,19
1,100000000146092,KJD00002,2-4 (ZWAVEL ZALF) 30GRAM - N/A,"SULFUR, SALICYLIC ACID",NORMAL,"3,31"
2,100000000146091,KJD00001,2-4 ZALF /GRAM - N/A,"SULFUR, SALICYLIC ACID",NORMAL,"3,5,31"
3,100000000142499,DT00146R,3TC 150MG TAB,LAMIVUDINE,NORMAL,"3,17,27,32,43"
4,117701,LVD00001,3TC-HBV 100MG TAB,LAMIVUDINE,NORMAL,"2,3,4,5,6,10,14,15,17,19,32,37,39"


In [5]:
df.Formularium.value_counts()

Formularium
NORMAL      6276
BPJS        2856
INHEALTH    1805
Name: count, dtype: int64

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10937 entries, 0 to 10936
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   SalesItemId            10937 non-null  int64 
 1   SalesItemCode          10937 non-null  object
 2   SalesItemName          10937 non-null  object
 3   ActiveIngredientsName  7257 non-null   object
 4   Formularium            10937 non-null  object
 5   OrganizationId         10937 non-null  object
dtypes: int64(1), object(5)
memory usage: 512.8+ KB


In [5]:
df['SalesItemId'].duplicated().sum()

np.int64(0)

In [6]:
df.sample(10)


,SalesItemId,SalesItemCode,SalesItemName,ActiveIngredientsName,Formularium,OrganizationId
9261,10835,D10835,RIFAMPICIN 300MG CAP,RIFAMPICIN,BPJS,"2,4,5,6,7,9,10,12,13,15,16,17,19,20,21,23,25,2..."
792,2000000253405,MCD01115,ANTI WRINKLE 10GRAM CREAM (DR MUTTY),NaN,NORMAL,19
6223,1000002919,D0000283,JS/ LIPOMED 20%-100ML INF,MEDIUM CHAIN TRIGLYCERIDES,BPJS,19
432,49000000276,AMD00222,am/ INTERVASK 5MG TAB,NaN,NORMAL,49
3807,1000006781,D0000448,in/ EFRISEL 10% 5ML EYE DROPS (CENDO),PHENYLEPHRINE,INHEALTH,3
3471,100000000131435,LVD01713,"in/ BACTESYN 1,5GRAM INJ",NaN,INHEALTH,"2,3,4,5,6,15,19,20,22,27,34,42,48"
10383,1000016057,D0000916,VACCINE INDOVAC 5ML INJ (DINKES),SARS-COV2 RBD (VACCINE COVID),BPJS,2
10630,11017,D11017,VITAMIN A 20.000UNIT TAB,RETINOL (VITAMIN A),BPJS,"5,7,9,14,15,17,19,20,28,29,32,33,36,43,51"
6085,17000000159,PLD00159,js/ INTRALIPID 100ML IV EMULSION FOR INF,LIPID,BPJS,"17,29,32,37"
403,49000000439,AMD00194,am/ HYPERRHO/BAYRHO D VIAL,NaN,NORMAL,49


In [7]:
df.shape


(10937, 6)

In [8]:
sales_data = pd.read_csv(PATH, sep=';').to_dict(orient='records')
sales_data[0]

{'SalesItemId': 19000000062,
 'SalesItemCode': 'MCD00062',
 'SalesItemName': '2 MTM BODY LOTION (DR MUTTY)',
 'ActiveIngredientsName': ' KOSMETIK',
 'Formularium': 'NORMAL',
 'OrganizationId': '19'}

In [9]:
# Initiate OpenAI embeddings.
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

url = "http://localhost:6333"
collection_name = "test_salesItem"
distance = Distance.COSINE
dimension = 1536 


In [12]:
# Initiate client.
client = QdrantClient("localhost", port=6333)

# Show collections.
collections = client.get_collections()
print(collections)


collections=[]


In [82]:
# Show collection details.
collection_info = client.get_collection(collection_name="test_salesItem")
print(collection_info)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=10 segments_count=4 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None

In [ ]:
# Delete collection.
# client.delete_collection(collection_name="test_salesItem")


True

In [ ]:
def moveEmbedding(sales_data, batch_size=1000):
    """
    Load sales items data from a list of dictionaries and upsert embeddings into Qdrant,
    using batch processing to speed up embedding generation.
    """
    # Initialize Qdrant client.
    client = qdrant_client.QdrantClient(url=url)
    
    # Create collection if it doesn't exist.
    if not client.collection_exists(collection_name=collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=dimension, distance=distance),
        )
    
    total_batches = (len(sales_data) - 1) // batch_size + 1
    for batch_num in range(total_batches):
        start = batch_num * batch_size
        end = start + batch_size
        batch_sales_data = sales_data[start:end]
        
        texts = []
        ids = []
        payloads = []
        
        # Build texts, IDs, and payloads for the current batch.
        for item in batch_sales_data:
            text = (
                "Sales Item Name: " + str(item['SalesItemName']) + "\n" +
                "Active Ingredients: " + str(item['ActiveIngredientsName']) + "\n" +
                "Formularium: " + str(item['Formularium'])
            )
            texts.append(text)
            ids.append(item['SalesItemId'])
            payloads.append({
                "page_content": text,
                "metadata": {
                    "SalesItemName": item['SalesItemName'],
                    "SalesItemCode": item['SalesItemCode'],
                    "ActiveIngredientsName": item['ActiveIngredientsName'],
                    "Formularium": item['Formularium'],
                    "OrganizationId": item['OrganizationId']
                }
            })
        
        # Batch embed the texts using the embed_documents method.
        batch_embeddings = embeddings.embed_documents(texts)
        
        # Prepare points for the current batch.
        points = []
        for j, emb in enumerate(batch_embeddings):
            points.append(
                PointStruct(
                    id=ids[j],
                    vector=emb,
                    payload=payloads[j]
                )
            )
        
        # Upsert the current batch of points into Qdrant.
        client.upsert(
            collection_name=collection_name,
            points=points
        )
        print(f"Processed batch {batch_num+1}/{total_batches}")
    
    print(f"Added: {len(sales_data)} Sales Items")


Batch embedding = 1m 8.8s

In [14]:
moveEmbedding(sales_data)

Processed batch 1/11
Processed batch 2/11
Processed batch 3/11
Processed batch 4/11
Processed batch 5/11
Processed batch 6/11
Processed batch 7/11
Processed batch 8/11
Processed batch 9/11
Processed batch 10/11
Processed batch 11/11
Added: 10937 Sales Items


In [15]:
client = QdrantClient(
    url=url,
)
qdrant = Qdrant(
    client,
    embeddings=embeddings,
    collection_name=collection_name,
    distance_strategy=distance,
)


/tmp/ipykernel_2316/2148830083.py:4: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.1.2 and will be removed in 0.5.0. Use :class:`~QdrantVectorStore` instead.
  qdrant = Qdrant(


In [16]:
qdrant.similarity_search_with_score(
    query="2 MTM BODY LOTION (DR MUTTY)",
    k=1
)


[(Document(metadata={'SalesItemName': '2 MTM BODY LOTION (DR MUTTY)', 'SalesItemCode': 'MCD00062', 'ActiveIngredientsName': ' KOSMETIK', 'Formularium': 'NORMAL', 'OrganizationId': '19', '_id': 19000000062, '_collection_name': 'test_salesItem'}, page_content='Sales Item Name: 2 MTM BODY LOTION (DR MUTTY)\nActive Ingredients:  KOSMETIK\nFormularium: NORMAL'),
  0.8227804)]

In [17]:
doc = qdrant.similarity_search_with_score('2 MTM BODY LOTION (DR MUTTY)', score_threshold=0.5, k=1)
for d in doc:
    print(d[0].metadata)

{'SalesItemName': '2 MTM BODY LOTION (DR MUTTY)', 'SalesItemCode': 'MCD00062', 'ActiveIngredientsName': ' KOSMETIK', 'Formularium': 'NORMAL', 'OrganizationId': '19', '_id': 19000000062, '_collection_name': 'test_salesItem'}


In [49]:
test_list = [{'query': x['SalesItemName'], 'expected': x['SalesItemName']} for x in sales_data[:10]]
print(test_list[0])

{'query': '2 MTM BODY LOTION (DR MUTTY)', 'expected': '2 MTM BODY LOTION (DR MUTTY)'}


In [50]:
test_list = test_list + [
    {
        "query": "vaksin campak dinkes",
        "expected": "VACCINE CAMPAK 4ML INJ (DINKES)"
    },
    {
        "query": "obh batuk berdahak",
        "expected": "OBH BATUK BERDAHAK 100ML SYR"
    },
    {
        "query": "detol cair",
        "expected": "DETTOL CAIR 100ML"
    },
    {
        "query": "antiseptic",
        "expected": "SABUN ANTISEPTIC A (DR.MUTTY)"
    },
    {
        "query": "sirup batuk pilek",
        "expected": "TRIAMINIC BATUK & PILEK 60ML SYR"
    },
    {
        "query": "test",
        "expected": "test"
    }
]


In [ ]:
def testing(qdrant, q, expected):
    doc = qdrant.similarity_search_with_score(q, score_threshold=0.1, k=10)
    i = 0
    for d in doc:
        if(d[0].metadata["SalesItemName"] == expected):
            return [ True, d]
        i += 1
    if len(doc) == 0:
        return [False, "No result"]
    return [False, doc[0]]


In [52]:
# With question only.
results = []
for t in test_list:
    [status, data] = testing(qdrant, t["query"], t["expected"])
    if(status == False):
        _data = {
            "query": t["query"],
            "result": data[0].metadata['SalesItemName'] if data != "No result" else "No result",
            "score": data[1] if data != "No result" else "No result"
        }
        results.append(_data)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    df = pd.DataFrame(data=results)
    display(df)
    

,query,result,score
0,test,AKURAT TESTPACK PCS -N/A,0.282302
